In [12]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 300)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split



#import matplotlib.pyplot as plt

#import seaborn as sns
#sns.set()

#import scipy as sp

#import statsmodels.api as sm
#import statsmodels.stats.api as sms

## Check DataFrame ##

In [2]:
base = pd.read_csv('../Data/BaseData.csv')

In [3]:
base.shape

(2580, 299)

In [4]:
base.isnull().sum().sum()

0

In [5]:
base.head(3)

,Unnamed: 0,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSewr,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_Po,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_None,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None,PoolQC_TA,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_CWD,SaleType_Con,SaleType_ConL

## Sample Cross Validation To Show Jonathan ##

In [6]:
# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold
# data sample
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# prepare cross validation
kfold = KFold(n_splits = 4, shuffle = True, random_state=None)
#kfold = KFold(3, True, 1)
# enumerate splits
for train, test in kfold.split(data):
    print('train: %s, test: %s' % (data[train], data[test]))


train: [0.1 0.2 0.3 0.6], test: [0.4 0.5]
train: [0.2 0.3 0.4 0.5], test: [0.1 0.6]
train: [0.1 0.3 0.4 0.5 0.6], test: [0.2]
train: [0.1 0.2 0.4 0.5 0.6], test: [0.3]


## Applying The Above CV Strategy To Our Case ##

In [7]:
X = base.drop(['SalePrice'], axis=1).values
y = base.SalePrice.values

#Other Input Combinations:
#X = base[['OverallQual']].values
#X = base[['QAT']].values


folds = KFold(n_splits = 5, shuffle = True, random_state = None)

for trainIndex, testIndex in folds.split(X):
    X_train, X_test = X[trainIndex], X[testIndex]
    y_train, y_test = y[trainIndex], y[testIndex]
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    print('R2 Train is: ' + str(model.score(X_train, y_train)))
    
    pred = model.predict(X_test)
    print('R2 Test is: ' + str(r2_score(y_test, pred)))
    
    ADJ_R2 = 1 - (1-r2_score(y_test, pred)) * (len(y)-1)/(len(y)-X.shape[1]-1)
    print('ADJ_R2 is: ' + str(ADJ_R2))
    
    RSS = pd.DataFrame({'Actual': y_test, 'Predicted':pred})
    #TSS = pd.DataFrame({'Actual': y_test, 'Average y':np.mean(y_test)})
    print('RSS is: '+ str(np.sum(np.square(RSS['Predicted'] - RSS['Actual']))))
    print('TSS is: '+ str(np.sum(np.square(np.mean(y_test) - RSS['Actual']))))
    print()
    
    
#result = next(kf.split(base), None)
#train = base.iloc[result[0]]
#test =  base.iloc[result[1]]

#print(len(train))
#print(len(test))

(2064, 298)
(2064,)
(516, 298)
(516,)
R2 Train is: 0.9393265914843513
R2 Test is: 0.9202528164396123
ADJ_R2 is: 0.9098342891704341
RSS is: 251072666263.67105
TSS is: 3148357785871.51

(2064, 298)
(2064,)
(516, 298)
(516,)
R2 Train is: 0.9392963270811494
R2 Test is: 0.9292142812165779
ADJ_R2 is: 0.9199665196219002
RSS is: 202750846963.06635
TSS is: 2864290289732.145

(2064, 298)
(2064,)
(516, 298)
(516,)
R2 Train is: 0.9498473960329562
R2 Test is: 0.8498713667647944
ADJ_R2 is: 0.8302578934179766
RSS is: 446548727415.39966
TSS is: 2974440769841.649

(2064, 298)
(2064,)
(516, 298)
(516,)
R2 Train is: 0.9432848895819677
R2 Test is: 0.9084926905601239
ADJ_R2 is: 0.8965377680642523
RSS is: 263486978157.69147
TSS is: 2879409085137.76

(2064, 298)
(2064,)
(516, 298)
(516,)
R2 Train is: 0.9468514741350723
R2 Test is: -14937.271428981183
ADJ_R2 is: -16888.873746314104
RSS is: 3.949590925706135e+16
TSS is: 2643941064053.5566



## R-Squared on Entire DataSet, Initial Model Assessment ##

In [8]:
def performance(data, scikit_model, select=None, chosen_list=None):
    
    y=data.SalePrice
    X=data.drop('SalePrice', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30)
    
    if chosen_list==None:
        X=data.drop('SalePrice', axis=1)
        
    else:
        X=data[chosen_list]
            
    
    if select!=None:
        sfs_foreward = SequentialFeatureSelector(scikit_model,
                                                 n_features_to_select = select,
                                                 direction="forward").fit(X,y)
        
        selected = X.columns[sfs_foreward.get_support()]
        
        model = scikit_model
        model.fit(X[selected],y)
        R2 = model.score(X[selected],y)
        
        Adj_R2 = 1 - (1-R2) * (len(y)-1)/(len(y)-X.shape[1]-1)
        
        RMSE = mean_squared_error(y, model.predict(X[selected]), squared=False)
        
        print(scikit_model)
        print("Number Of Features: %.d" %(select))
        print("Features are: %s" %(sfs_foreward.get_feature_names_out()))
    #    print(type(sfs_foreward.get_feature_names_out()))
        print("R-Squared is: %.6f" %(R2))
        print("Adjusted R-Squared is: %.6f" %(Adj_R2))
    #    print("RMSE in US$ is: %.2f" %(RMSE))
 

    else:  
        model = scikit_model
        model.fit(X,y)
        R2 = model.score(X,y)
        Adj_R2 = 1 - (1-R2) * (len(y)-1)/(len(y)-X.shape[1]-1)
        RMSE = mean_squared_error(y, model.predict(X), squared=False)
        
        model_train = scikit_model
        model_train.fit(X_train, y_train)
        R2_train = model_train.score(X_train,y_train)
        RMSE_train = mean_squared_error(y_train, 
                                        model_train.predict(X_train),
                                        squared=False)
        
        preds = model_train.predict(X_test)
        R2_test = r2_score(y_test, preds)
        RMSE_test = mean_squared_error(y_test, preds, squared=False)
               
        print(scikit_model)
        print("Number Of Features: %.d" %(len(X.columns)))
    #    print()
    #    print("Results on Entire Data:")    
        print("R-Squared is: %.6f" %(R2))
        print("Adjusted R-Squared is: %.6f" %(Adj_R2))
    #    print("RMSE in US$ is: %.2f" %(RMSE))
        print()
        
    #   perc_train = f"{0.7:.0%}"
     #   print("Results on Training Data (" + perc_train + "):")
     #   print("R-Squared is: %.6f" %(R2_train))
     #   print("RMSE in US$ is %.2f" %(RMSE_train))
     #   print()
        
     #   perc_test = f"{0.3:.0%}"
     #   print("Results on Test Data (" + perc_test + "):")
     #   print("R-Squared is: %.6f" %(R2_test))
     #   print("RMSE in US$ is %.2f" %(RMSE_test))
            

## Sample Model Performance w/o CV ##

In [9]:
performance(base, LinearRegression())

LinearRegression()
Number Of Features: 298
R-Squared is: 0.939686
Adjusted R-Squared is: 0.931806



In [10]:
performance(base, Ridge())

Ridge()
Number Of Features: 298
R-Squared is: 0.938051
Adjusted R-Squared is: 0.929958



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=6.67842e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=8.78338e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [13]:
performance(base, Lasso())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.464e+11, tolerance: 1.452e+09
  model = cd_fast.enet_coordinate_descent(


Lasso()
Number Of Features: 298
R-Squared is: 0.939604
Adjusted R-Squared is: 0.931713



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.786e+11, tolerance: 1.104e+09
  model = cd_fast.enet_coordinate_descent(


In [14]:
performance(base, Ridge(alpha=37.82))

Ridge(alpha=37.82)
Number Of Features: 298
R-Squared is: 0.923018
Adjusted R-Squared is: 0.912960



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.71426e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=2.26707e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [10]:
performance(base, LinearRegression(), select=1)

LinearRegression()
Number Of Features: 1
Features are: ['OverallQual']
R-Squared is: 0.624893
Adjusted R-Squared is: 0.575887


In [11]:
performance(base, LinearRegression(), chosen_list=['GrLivArea'])

LinearRegression()
Number Of Features: 1
R-Squared is: 0.518373
Adjusted R-Squared is: 0.518186



In [12]:
performance(base, LinearRegression(), select=2)

LinearRegression()
Number Of Features: 2
Features are: ['GrLivArea' 'OverallQual']
R-Squared is: 0.736967
Adjusted R-Squared is: 0.702603


In [13]:
performance(base, LinearRegression(), select=3)

LinearRegression()
Number Of Features: 3
Features are: ['GrLivArea' 'OverallQual' 'BsmtFinSF1']
R-Squared is: 0.799004
Adjusted R-Squared is: 0.772745


In [14]:
performance(base, LinearRegression(), select=4)

LinearRegression()
Number Of Features: 4
Features are: ['GrLivArea' 'OverallQual' 'BsmtFinSF1' 'GarageArea']
R-Squared is: 0.814659
Adjusted R-Squared is: 0.790445


In [15]:
performance(base, LinearRegression(), select=5)

LinearRegression()
Number Of Features: 5
Features are: ['GrLivArea' 'OverallQual' 'BsmtFinSF1' 'TotalBsmtSF' 'GarageArea']
R-Squared is: 0.826196
Adjusted R-Squared is: 0.803490


In [16]:
performance(base, LinearRegression(), select=6)

LinearRegression()
Number Of Features: 6
Features are: ['GrLivArea' 'OverallQual' 'YearRemodAdd' 'BsmtFinSF1' 'TotalBsmtSF'
 'GarageArea']
R-Squared is: 0.834549
Adjusted R-Squared is: 0.812934


In [17]:
performance(base, LinearRegression(), select=7)

LinearRegression()
Number Of Features: 7
Features are: ['BsmtExposure_Gd' 'GrLivArea' 'OverallQual' 'YearRemodAdd' 'BsmtFinSF1'
 'TotalBsmtSF' 'GarageArea']
R-Squared is: 0.840769
Adjusted R-Squared is: 0.819967


In [18]:
performance(base, LinearRegression(), select=8)

LinearRegression()
Number Of Features: 8
Features are: ['Neighborhood_NridgHt' 'BsmtExposure_Gd' 'GrLivArea' 'OverallQual'
 'YearRemodAdd' 'BsmtFinSF1' 'TotalBsmtSF' 'GarageArea']
R-Squared is: 0.847394
Adjusted R-Squared is: 0.827457


In [19]:
performance(base, LinearRegression(), select=9)

LinearRegression()
Number Of Features: 9
Features are: ['Neighborhood_NoRidge' 'Neighborhood_NridgHt' 'BsmtExposure_Gd'
 'GrLivArea' 'OverallQual' 'YearRemodAdd' 'BsmtFinSF1' 'TotalBsmtSF'
 'GarageArea']
R-Squared is: 0.852785
Adjusted R-Squared is: 0.833553


In [20]:
performance(base, LinearRegression(), select=10)

LinearRegression()
Number Of Features: 10
Features are: ['MSSubClass_20' 'Neighborhood_NoRidge' 'Neighborhood_NridgHt'
 'BsmtExposure_Gd' 'GrLivArea' 'OverallQual' 'YearRemodAdd' 'BsmtFinSF1'
 'TotalBsmtSF' 'GarageArea']
R-Squared is: 0.857051
Adjusted R-Squared is: 0.838375


In [21]:
performance(base, Ridge(alpha=0.00001))

Ridge(alpha=1e-05)
Number Of Features: 298
R-Squared is: 0.939686
Adjusted R-Squared is: 0.931806



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=8.17015e-26): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.16678e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [22]:
performance(base, Ridge(alpha=115.5))

Ridge(alpha=115.5)
Number Of Features: 298
R-Squared is: 0.912999
Adjusted R-Squared is: 0.901633



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=5.21873e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=7.31819e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


### Ideas for Feature Engineering ###

In [23]:
performance(base, LinearRegression(), 
            chosen_list=['GrLivArea', 'TotalBsmtSF', 'GarageArea',
                         'MasVnrArea', 'WoodDeckSF','LotFrontage','OpenPorchSF'])

LinearRegression()
Number Of Features: 7
R-Squared is: 0.741061
Adjusted R-Squared is: 0.740356



In [24]:
GBGVWLO = base[['GrLivArea', 'TotalBsmtSF', 'GarageArea','MasVnrArea', 'WoodDeckSF','LotFrontage','OpenPorchSF']]
GBGVWLO.shape

(2580, 7)

In [25]:
base = base.assign(GBGVWLO = GBGVWLO['GrLivArea'] + GBGVWLO['TotalBsmtSF'] + 
                     GBGVWLO['GarageArea'] + GBGVWLO['MasVnrArea'] + GBGVWLO['WoodDeckSF'] +
                      GBGVWLO['LotFrontage'] + GBGVWLO['OpenPorchSF'])
base.head(3)

,Unnamed: 0,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSewr,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_Po,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_None,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None,PoolQC_TA,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_CWD,SaleType_Con,SaleType_ConL

In [26]:
performance(base, LinearRegression(), chosen_list=['GBGVWLO'])

LinearRegression()
Number Of Features: 1
R-Squared is: 0.738690
Adjusted R-Squared is: 0.738589



In [27]:
base = base.assign(QA = base['OverallQual']*base['GrLivArea'])

In [28]:
performance(base, LinearRegression(), chosen_list=['QA'])

LinearRegression()
Number Of Features: 1
R-Squared is: 0.745924
Adjusted R-Squared is: 0.745826



In [29]:
base = base.assign(QAT = base['GBGVWLO']*base['OverallQual'])

In [30]:
base.head()

,Unnamed: 0,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSewr,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_Po,...,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_None,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None,PoolQC_TA,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,Sal

In [31]:
#Quality x Total Area, simple & elegant. 0.84 R2. 
performance(base, LinearRegression(), chosen_list=['QAT'])

LinearRegression()
Number Of Features: 1
R-Squared is: 0.846882
Adjusted R-Squared is: 0.846823



In [32]:
n = base.select_dtypes('number')
n.shape

(2580, 302)

In [36]:
performance(base, HuberRegressor(), chosen_list=['QAT'])

HuberRegressor()
Number Of Features: 1
R-Squared is: 0.762062
Adjusted R-Squared is: 0.761969

